# Miniproyecto 1 (Actividad 3)
---

##### Integrantes:
<i> - Hugo Torricos
<br><i> - Alejandro Tolosa
<br><i> - Isabel Catalán
<br><i> - Anderson Suárez

---
### Transformación e imputación de datos 

Abrir entorno de programación, de preferencia utilizar Visual studio code. Importe las librerías pandas, searborn, matplotlib, numpy y sklearn. Le recomendamos usar un ambiente de conda específico para el curso.

In [2]:
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

Cargar la base de datos (Gaia NaN.csv o metadato.csv ). Cree una función que permita cargar la base de datos bajo diferentes condiciones. Los argumentos de esta función deben ser: (i) un string con el nombre del directorio donde se encuetre la base de datos, (ii) una variable booleana que indique si se trabajará con una muestra o con la base de datos completa y (iii) un argumento que reciba las columnas con las que se pueda trabajar en una lista. Usted puede agregar nuevos argumentos que den mayor flexibilidad a la carga de datos. Recuerde verificar el tipo de variable reconocido por pandas.

In [ ]:
def loadData(folder:str, sample=False, useColumns=[], samplesize=-1):

    df = pd.read_csv(folder)
    if sample:
        df = df.loc[0:samplesize-1]
    for c in df.columns:
        if c not in useColumns:
            df = df.drop(c, axis=1)
    
    return df

Genere un diagnóstico de estadística descriptiva y de datos faltantes. Cree una función que permita realizar el diagnóstico de forma flexible, la función debe retornar, media, desviación estándar, valores perdidos por descriptor, valor máximo y valor mínimo. Usted puede usar funciones internas de otras librerías. Cada uno de los estadísticos debe ser un argumento booleano en la función y solo cuando se indique True este se calculará. Los descriptores para los cuales se calcular´an estos descriptores también deben ser un argumento de la función.

In [ ]:
def diagnosis(db, mean=True, stdDev=True, lostValues=True, maxVal=True, minVal=True, descriptors=[]):
    if mean:
        for d in descriptors:
            print(f"{d} mean: {db[d].mean()}")
    if stdDev:
        for d in descriptors:
            print(f"{d} standard deviation: {db[d].std()}")
    if lostValues:
        for d in descriptors:
            print(f"{d} na count: {len(db)-db[d].count()}")
    if maxVal:
        for d in descriptors:
            print(f"{d} max: {db[d].max()}")
    if minVal:
        for d in descriptors:
            print(f"{d} min: {db[d].min()}")

Impute los datos perdidos con el método de su elección. Genere una función que reciba una lista de descriptores, el dataframe original y una lista con las estrategias de imputación de cada descriptor. La función debe retornar la nueva base de datos imputada. ¿Cómo cambió la distribución de los datos con la imputación realizada?

In [ ]:
def fillMissing(db, descriptors=[], strats=[]):
    for i in range(descriptors):
        if strats[i].split[0]=='fill':
            if strats[i].split[1] == 'True':
                db[descriptors[i]] = db[descriptors[i]].fillna(True)
            elif strats[i].split[1] == 'False':
                db[descriptors[i]] = db[descriptors[i]].fillna(False)
            elif strats[i].split[1].replace('.','',1).isdigit():
                db[descriptors[i]] = db[descriptors[i]].fillna(float(strats[i].split[1]))
            else:
                db[descriptors[i]] = db[descriptors[i]].fillna(strats[i].split[1])
        elif strats[i].split[0]=='ffill':
            db[descriptors[i]] = db[descriptors[i]].ffill()
        elif strats[i].split[0]=='bfill':
            db[descriptors[i]] = db[descriptors[i]].bfill()
    return db

Genera 2 gráficos diferentes (ejemplo: boxplot, scatter plot, histogramas, gráfico de torta, etc.) que entreguen información relevante para el modelamiento del problema (ejemplo: correlaciones evidentes, datos at´ıpicos, patrones no lineales de relaciones, etc). Debe explicar tanto la elecci´on de cada gráfico como la información otenida a partir de ellos.

Cree una función que permita hacer scatter plots y/o box plots para dos descriptores datos. La función debe recibir como argumento las dos variables, y el tipo de gráfico que se desea obtener. La función debe recibir como argumento la decisión de visualizar o guardar los gráficos realizados. Usted puede agregar más argumentos para obtener visualizaciones más personalizadas. Usando dicha función, genere visualizaciones para 5 de los descriptores de la base de datos entregada.

In [ ]:
def graph(db, xVar, yVar, graphType, save = False):
    plt.figure(figsize=[])
    plt.title(graphType)
    
    if graphType == "scatter":
        plt.xlabel(xVar)
        plt.ylabel(yVar)
        plt.scatter(db[xVar], db[yVar], c='#008080', alpha=0.5)
    elif graphType == "boxplot":
        plt.xlabel("Categorías")
        plt.ylabel("Valores")
        plt.boxplot(db[xVar], db[yVar], vert=True, patch_artist=True)
        plt.xticks([1, 2], [xVar, yVar])
    
    if save:
        plt.savefig("imagen1.jpg")
    else:
        plt.show()

In [ ]:
# Graficos

Aplique normalización z o escalamiento a los datos. Genere una función que permita aplicar estas transformaciones a los datos, como argumento se debe indicar qué tipo de estrategia se usará para cada descriptor. La función debe retornar el dataframe modificado.

In [ ]:
def processDescriptors(db, strats=[]):
    for s in strats:
        if s.split()[1] == 'zscore':
            db[s.split()[0]].apply(zscore)
        elif s.split()[1] == 'escalamiento':
            scaler = MinMaxScaler()
            db[s.split()[0]] = scaler.fit(db[[s.split()[0]]])
    return db

Genere sets de entrenamiento y testeo, con separación estratificada. Genere una función que aplique este procesamiento. No olvide fijar la semilla aleatoria para poder replicar los resultados.

In [ ]:
def trainAndTestSets(db):
    X = db.loc[:, :-1]
    y = db.loc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=10, stratify=StratifiedKFold)
    return (X_train, X_test, y_train, y_test)

Consolide todas las funciones en una clase. Esta clase tendrá por nombre preprocesamiento. Algunos de los parámetros que se usan en las funciones antes creadas pueden ser entregadas en la inicialización de la clase. Agregue una función que aplique todo el procesamiento, denomine a esta función ejecutar procesamiento.

In [ ]:
class preprocesamiento():
    def __init__(self, data):
        self.data = data

    def loadData(self, folder:str, sample=False, useColumns=[], samplesize=-1):
        df = pd.read_csv(folder)
        if sample:
            df = df.loc[0:samplesize-1]
        for c in df.columns:
            if c not in useColumns:
                df = df.drop(c, axis=1)
        return df

    def diagnosis(self, db, mean=True, stdDev=True, lostValues=True, maxVal=True, minVal=True, descriptors=[]):
        if mean:
            for d in descriptors:
                print(f"{d} mean: {db[d].mean()}")
        if stdDev:
            for d in descriptors:
                print(f"{d} standard deviation: {db[d].std()}")
        if lostValues:
            for d in descriptors:
                print(f"{d} na count: {len(db)-db[d].count()}")
        if maxVal:
            for d in descriptors:
                print(f"{d} max: {db[d].max()}")
        if minVal:
            for d in descriptors:
                print(f"{d} min: {db[d].min()}")
        

    def fillMissing(self, db, descriptors=[], strats=[]):
        for i in range(descriptors):
            if strats[i].split[0]=='fill':
                if strats[i].split[1] == 'True':
                    db[descriptors[i]] = db[descriptors[i]].fillna(True)
                elif strats[i].split[1] == 'False':
                    db[descriptors[i]] = db[descriptors[i]].fillna(False)
                elif strats[i].split[1].replace('.','',1).isdigit():
                    db[descriptors[i]] = db[descriptors[i]].fillna(float(strats[i].split[1]))
                else:
                    db[descriptors[i]] = db[descriptors[i]].fillna(strats[i].split[1])
            elif strats[i].split[0]=='ffill':
                db[descriptors[i]] = db[descriptors[i]].ffill()
            elif strats[i].split[0]=='bfill':
                db[descriptors[i]] = db[descriptors[i]].bfill()
        return db

    def graph(self, db, xVar, yVar, graphType, save = False):
        plt.figure(figsize=[])
        if graphType == "scatter":
            plt.title("")
            plt.xlabel(xVar)
            plt.ylabel(yVar)
            plt.scatter(db[xVar], db[yVar], c='#008080', alpha=0.5)
        elif graphType == "boxplot":
            plt.title("")
            plt.xlabel("Categorías")
            plt.ylabel("Valores")
            plt.boxplot(db[xVar], db[yVar], vert=True, patch_artist=True)
            plt.xticks([1, 2], [xVar, yVar])
        if save:
            plt.savefig("imagen1.jpg")
        else:
            plt.show()
        
    def processDescriptors(self, db, strats=[]):
        for s in strats:
            if s.split()[1] == 'zscore':
                db[s.split()[0]].apply(zscore)
            elif s.split()[1] == 'escalamiento':
                scaler = MinMaxScaler()
                db[s.split()[0]] = scaler.fit(db[[s.split()[0]]])
        return db
    
    def trainAndTestSets(self, db):
        X = db.loc[:, :-1]
        y = db.loc[:, -1]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=10, stratify=StratifiedKFold)
        return (X_train, X_test, y_train, y_test)
    
    def ejecutar_procesamiento(self, prep, file):

        data = prep.loadData(file)
        prep.diagnosis(data, descriptors=['age'])
        data = prep.fillMissing(data, descriptors=['smoke'], strats=['fill False'])
        prep.graph()
        prep.graph()
        prep.processDescriptors()
        prep.data = data
        return prep.trainAndTestSets(data)

In [ ]:
ruta = 'data\measurements.csv'
prep = preprocesamiento(ruta)

prep.ejecutar_procesamiento(prep, ruta)

---
### Entrenamiento de modelos 

Ajuste los clasificadores naive Bayes (desde sklearn.naive bayes.GaussianNB) y regresión logística (desde sklearn.linear model.LogisticRegression ). Genere una función con nombre clasificador que reciba como argumento: (i) el tipo de clasificador que desea ajustar, (ii) el nombre de la dirección donde se guardara el modelo y (iii) los datos de entrenamiento. La función solo debe ajustar y guardar el modelo.

Cree una función que tenga por nombre evaluar rendimiento, esta función debe recibir la direcci´on del modelo, los datos que desea evaluar (entrenamiento o test) y el tipo de análisis. Los análisis posibles son: (i) mostrar la matriz de confusión y (ii) mostrar las métricas de evaluación (accuracy, recall, precision y F1-score).

Use estas funciones para probar distintos modelos, explore los siguientes argumentos en la regresión logística: penalty, C, class weight, l1 ratio. En naive Bayes modifique: priors de acuerdo a la descripción de la librería. Entregue un análisis de los resultados y seleccione un modelo. También puede aplicar procedimientos para seleccionar los descriptores que se incluyen en el modelo final.

---
### Modelos basados en árboles

a) Explore la documentación del modelo clasificador árbol de decisión link. Describa el aprendizaje de este modelo.

b) Ajuste un árbol de decisión a los datos usados en las secciones anteriores y compare su rendimiento con respecto a los modelos naive Bayes y la regresión logística

c) Repita a y b pero con el clasificador random forest link

### Transformación e Imputación de Datos

In [ ]:
def ejecutar_procesamiento(prep, file):

    data = prep.loadData(file)

    prep.diagnosis(data, descriptors=['age'])

    data = prep.fillMissing(data, descriptors=['smoke'], strats=['fill False'])

    prep.graph()

    prep.graph()

    prep.processDescriptors()

    prep.data = data

    return prep.trainAndTestSets(data)

In [ ]:
ruta = 'data\measurements.csv'
prep = preprocesamiento([])
ejecutar_procesamiento(prep, ruta)

### Entrenamiento de modelos

In [ ]:
def clasificador(type, saveTo, trainingData):
    pass

In [ ]:
def evaluar_rendimiento(folder:str, data, analysisType):
    pass